In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("asaniczka/top-spotify-songs-in-73-countries-daily-updated")

print("Path to dataset files:", path)

C:\Users\nurka\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 121M/121M [00:40<00:00, 3.12MB/s] 

Extracting files...


Path to dataset files: C:\Users\nurka\.cache\kagglehub\datasets\asaniczka\top-spotify-songs-in-73-countries-daily-updated\versions\457


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
df = pd.read_csv(path + '/universal_top_spotify_songs.csv')

df.head()

,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity,is_explicit,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2plbrEY59IikOBgBGLjaoe,Die With A Smile,"Lady Gaga, Bruno Mars",1,0,0,NaN,2025-01-04,100,False,...,6,-7.777,0,0.0304,0.30800,0.0000,0.122,0.535,157.969,3
1,4wJ5Qq0jBN4ajy7ouZIV1c,APT.,"ROSÉ, Bruno Mars",2,0,0,NaN,2025-01-04,88,False,...,0,-4.477,0,0.2600,0.02830,0.0000,0.355,0.939,149.027,4
2,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,Billie Eilish,3,0,0,NaN,2025-01-04,97,False,...,2,-10.171,1,0.0358,0.20000,0.0608,0.117,0.438,104.978,4
3,7ne4VBA60CxGM75vw0EYad,That’s So True,Gracie Abrams,4,0,0,NaN,2025-01-04,96,True,...,1,-4.169,1,0.0368,0.21400,0.0000,0.159,0.372,108.548,4
4,7tI8dRuH2Yc6RuoTjxo4dU,Who,Jimin,5,0,0,NaN,2025-01-04,91,False,...,0,-3.743,0,0.0320,0.00289,0.0000,0.193,0.838,116.034,4


In [4]:
df.columns

Index(['spotify_id', 'name', 'artists', 'daily_rank', 'daily_movement',
       'weekly_movement', 'country', 'snapshot_date', 'popularity',
       'is_explicit', 'duration_ms', 'album_name', 'album_release_date',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature'],
      dtype='object')

In [5]:
df['country'].unique()

array([nan, 'ZA', 'VN', 'VE', 'UY', 'US', 'UA', 'TW', 'TR', 'TH', 'SV',
       'SK', 'SG', 'SE', 'SA', 'RO', 'PY', 'PT', 'PL', 'PK', 'PH', 'PE',
       'PA', 'NZ', 'NO', 'NL', 'NI', 'NG', 'MY', 'MX', 'MA', 'LV', 'LU',
       'LT', 'KZ', 'KR', 'JP', 'IT', 'IS', 'IN', 'IL', 'IE', 'ID', 'HU',
       'HN', 'HK', 'GT', 'GR', 'FR', 'FI', 'ES', 'EG', 'EE', 'EC', 'DO',
       'DK', 'DE', 'CZ', 'CR', 'CO', 'CL', 'CH', 'CA', 'BY', 'BR', 'BO',
       'BG', 'BE', 'AU', 'AT', 'AR', 'AE', 'GB'], dtype=object)

In [6]:
df = df[(df['country'] == "ID") | (df['country'] == "US") | (df['country'].isna())]
df.drop(['country'], axis=1, inplace=True)

df.head()

,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,snapshot_date,popularity,is_explicit,duration_ms,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2plbrEY59IikOBgBGLjaoe,Die With A Smile,"Lady Gaga, Bruno Mars",1,0,0,2025-01-04,100,False,251667,...,6,-7.777,0,0.0304,0.30800,0.0000,0.122,0.535,157.969,3
1,4wJ5Qq0jBN4ajy7ouZIV1c,APT.,"ROSÉ, Bruno Mars",2,0,0,2025-01-04,88,False,169917,...,0,-4.477,0,0.2600,0.02830,0.0000,0.355,0.939,149.027,4
2,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,Billie Eilish,3,0,0,2025-01-04,97,False,210373,...,2,-10.171,1,0.0358,0.20000,0.0608,0.117,0.438,104.978,4
3,7ne4VBA60CxGM75vw0EYad,That’s So True,Gracie Abrams,4,0,0,2025-01-04,96,True,166300,...,1,-4.169,1,0.0368,0.21400,0.0000,0.159,0.372,108.548,4
4,7tI8dRuH2Yc6RuoTjxo4dU,Who,Jimin,5,0,0,2025-01-04,91,False,170887,...,0,-3.743,0,0.0320,0.00289,0.0000,0.193,0.838,116.034,4


In [7]:
print(df.isna().sum())

df.dropna(axis=0, inplace=True)

spotify_id             0
name                   0
artists                0
daily_rank             0
daily_movement         0
weekly_movement        0
snapshot_date          0
popularity             0
is_explicit            0
duration_ms            0
album_name            67
album_release_date    67
danceability           0
energy                 0
key                    0
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
time_signature         0
dtype: int64


In [8]:
df = df.drop_duplicates(subset=['spotify_id'])

In [9]:
df = df.drop_duplicates(subset=['name', 'artists'], keep='first')

df = df.reset_index(drop=True)

In [10]:
num_features = [
    "danceability",
    "energy",
    'key',
    "loudness",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
]

In [11]:
scaler = StandardScaler()

df[num_features] = scaler.fit_transform(df[num_features])

In [12]:
features = [
    'name',
    'artists',
    'popularity',
    'danceability',
    'energy',
    'loudness',
    'duration_ms',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'album_release_date'
]

df['combined_features'] = df[features].apply(lambda x: ' '.join(x.astype(str)), axis=1)

In [13]:
tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(df['combined_features'])

In [14]:
similarity = cosine_similarity(tfidf_matrix)

In [15]:
def recommend(song_id, similarity, df, top_n=10):
    song_idx = df.index[df['spotify_id'] == song_id][0]
    similar_indices = similarity[song_idx].argsort()[::-1][1:top_n+1]
    recommendations = df.iloc[similar_indices]
    return recommendations[['spotify_id', 'name', 'artists']]

In [16]:
recommendations = recommend('2plbrEY59IikOBgBGLjaoe', similarity, df, 20)

print(recommendations)

                 spotify_id                  name  \
287  19KlZwqlT3fguP2BeHF1Q1               Disease   
909  7mXuWTczZNxG5EDcjFEuJR             LADY GAGA   
310  11BKm0j4eYoCPPpCONAVwA           Bloody Mary   
1    4wJ5Qq0jBN4ajy7ouZIV1c                  APT.   
44   0KKkJNfGyhkQ5aFogxQAPU    That's What I Like   
367  161DnLWsx1i3u1JT05lzqU   Talking to the Moon   
317  4rMNgxEwGWoPHIUcvB5BrB          It Will Rain   
365  0B7wvvmu9EISAwZnOpjhNI   When I Was Your Man   
28   3w3y8KPTfNeOKPiqUTakBh  Locked out of Heaven   
41   2osJCvi2zzebMQTc7vvcAm  Just the Way You Are   
398  09LrGvT9KsACH66RHYMDyR           Don’t Smile   
599  5og4Qzt92jJzVDkOtSEilb  But Daddy I Love Him   
577  6IwmDRpswDujeciIBTiEOL            love hotel   
455  7Lfwb4IjZiNYGuqtP65kIc                 FRIED   
985  7tuwsUy09EoabNtrXrLnX1            Surat Hati   
821  0FlwhvrncUKrEAhzunmCKm               Forever   
592  7DpUoxGSdlDHfqCYj0otzU           BITTERSUITE   
37   3MJ6lFNV12FSHAOzTzkwxz                 Sm

In [73]:
X = df[num_features].values
y = df['album_release_date'].values

In [74]:
def triplet_loss(margin=.5):
    def loss(y_true, y_pred):
        anchor, positive, negative = tf.split(y_pred, num_or_size_splits=3, axis=1)

        positive_distance = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        negative_distance = tf.reduce_sum(tf.square(anchor - negative), axis=1)

        basic_loss = positive_distance - negative_distance + margin
        loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0))

        return loss

    return loss

In [75]:
def create_triplet(data, labels):
    triplets = []
    unique_labels = np.unique(labels)

    for label in unique_labels:
        positive = np.where(labels == label)[0]
        negative = np.where(labels != label)[0]

        for anchor in positive:
            positive_idx = np.random.choice(positive)
            negative_idx = np.random.choice(negative)
            triplets.append((data[anchor], data[positive_idx], data[negative_idx]))

    return np.array(triplets)

In [76]:
triplets = create_triplet(X, y)
anchor, positive, negative = zip(*triplets)

anchor = np.array(anchor)
positive = np.array(positive)
negative = np.array(negative)

In [77]:
def build_encoder():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(X.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(X.shape[1], activation='linear'),
        tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))
    ])

    return model

encoder = build_encoder()

In [78]:
anchor_input = tf.keras.layers.Input(shape=(X.shape[1],))
positive_input = tf.keras.layers.Input(shape=(X.shape[1],))
negative_input = tf.keras.layers.Input(shape=(X.shape[1],))

anchor_embedding = encoder(anchor_input)
positive_embedding = encoder(positive_input)
negative_embedding = encoder(negative_input)

output = tf.keras.layers.Concatenate()([anchor_embedding, positive_embedding, negative_embedding])

model = tf.keras.Model(inputs=[anchor_input, positive_input, negative_input], outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-3), loss=triplet_loss(margin=1e-3))
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_17      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_18      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_19      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_3        │ (None, 10)        │      3,114 │ input_layer_17[0… │
│ (Sequential)        │                   │            │ input_layer_18[0… │
│                     │                   │            │ input_layer_19[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 30)        │          0 │ sequential_3[0][… │
│ (Concatenate)       │                   │            │ sequential_3[1][… │
│                     │                   │            │ sequential_3[2][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,114 (12.16 KB)

 Trainable params: 3,114 (12.16 KB)

 Non-trainable params: 0 (0.00 B)

In [79]:
model.fit([anchor, positive, negative], np.zeros((anchor.shape[0], 1)), epochs=10, batch_size=256)

Epoch 1/10


C:\Users\nurka\AppData\Roaming\Python\Python312\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_57', 'keras_tensor_58', 'keras_tensor_59']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0668
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0100  
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0046
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0025  
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0016  
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0013  
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.7676e-04  
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.4913e-04
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.0337e-04
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.8119e-04


In [80]:
embeddings = model.predict([anchor, positive, negative])

print(embeddings)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
[[-0.04760594  0.2941113  -0.4347039  ...  0.02705927  0.13509037
   0.2227947 ]
 [-0.06862333  0.27812204 -0.4220403  ...  0.00163023  0.1069097
   0.20796688]
 [-0.07004015  0.3097249  -0.3995325  ... -0.00509571  0.05887238
   0.2557276 ]
 ...
 [-0.05177833  0.26716903 -0.4257603  ...  0.02979751  0.14539088
   0.23633115]
 [-0.05386121  0.31895772 -0.44168338 ...  0.00855023  0.12107926
   0.20275712]
 [-0.07467656  0.3234624  -0.392991   ...  0.00855533  0.13406551
   0.23780464]]


In [81]:
embeddings.shape

(1044, 30)

In [82]:
model_similarity = cosine_similarity(embeddings)

In [83]:
recommendations = recommend('2plbrEY59IikOBgBGLjaoe', model_similarity, df, 20)

print(recommendations)

                  spotify_id  \
11    3QaPy1KgI7nu9FJEQUgn6h   
9     35ISBknsCeZQtq66xABI9g   
10    0WbMK4wrZ1wFSty9F7FCgu   
120   1KZG1H1OCugGqIRmk1wF9m   
242   2HBBM75Xv3o2Mqdyh1NcM0   
621   5g7TAIGfB6vgbaX8AH19FG   
443   2hHeGD57S0BcopfVcmehdl   
1027  5IhULiOyG58jL20qWkztEg   
67    040I32EKLxQrkuxQu1pqvT   
444   0DM63CJEklNPKu6kHGyuEl   
126   4OLT65TRsOx3Iv1TlCcQVb   
1     4wJ5Qq0jBN4ajy7ouZIV1c   
6     2CGNAOSuO1MEFCbBRgUzjd   
238   0VaeksJaXy5R1nvcTMh3Xk   
215   3vkCueOmm7xQDoJ17W1Pm3   
829   4E3XDKil4ElJt6sXuz5cc6   
793   7aqfrAY2p9BUSiupwk3svU   
311   2dCmGcEOQrMQhMMS8Vj7Ca   
183   2OQ6a4CfUeYskpTTgyawyJ   
244   5RePVWy39tLpHH0WwXgBsK   

                                                   name  \
11                                           WILDFLOWER   
9                                                 Messy   
10                                     Good Luck, Babe!   
120                           Indigo (feat. Avery Anna)   
242                             

In [84]:
results = pd.concat([df['spotify_id'], pd.DataFrame(model_similarity)], axis=1)

results.head()

,spotify_id,0,1,2,3,4,5,6,7,8,...,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043
0,2plbrEY59IikOBgBGLjaoe,1.000000,0.998363,0.995436,0.997044,0.997750,0.994918,0.998356,0.997701,0.995141,...,0.997361,0.997755,0.995053,0.997820,0.997512,0.993936,0.995681,0.997521,0.992653,0.995945
1,4wJ5Qq0jBN4ajy7ouZIV1c,0.998363,1.000000,0.996643,0.996075,0.998716,0.995840,0.997953,0.997324,0.997928,...,0.998293,0.997541,0.995734,0.996966,0.998791,0.993573,0.994087,0.995774,0.992171,0.995849
2,6dOtVTDdiauQNBQEDOtlAB,0.995436,0.996643,1.000000,0.992924,0.995463,0.993506,0.995443,0.995666,0.992726,...,0.995585,0.994504,0.993810,0.995548,0.997187,0.992091,0.993265,0.994269,0.990478,0.995701
3,7ne4VBA60CxGM75vw0EYad,0.997044,0.996075,0.992924,1.000000,0.995819,0.996249,0.996445,0.996336,0.995173,...,0.995965,0.995526,0.990803,0.995055,0.995708,0.993931,0.992888,0.994538,0.987166,0.992802
4,7tI8dRuH2Yc6RuoTjxo4dU,0.997750,0.998716,0.995463,0.995819,1.000000,0.994088,0.996669,0.996609,0.996197,...,0.995796,0.995749,0.994016,0.996536,0.997809,0.993367,0.993848,0.995733,0.989868,0.992378


In [85]:
results.to_csv('results.csv', index=False)

In [86]:
model.save('model.keras')

In [87]:
database = df[['spotify_id', 'name', 'artists', 'album_name', 'duration_ms', 'album_release_date']]

database.head()

,spotify_id,name,artists,album_name,duration_ms,album_release_date
0,2plbrEY59IikOBgBGLjaoe,Die With A Smile,"Lady Gaga, Bruno Mars",Die With A Smile,251667,2024-08-16
1,4wJ5Qq0jBN4ajy7ouZIV1c,APT.,"ROSÉ, Bruno Mars",rosie,169917,2024-12-06
2,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,Billie Eilish,HIT ME HARD AND SOFT,210373,2024-05-17
3,7ne4VBA60CxGM75vw0EYad,That’s So True,Gracie Abrams,The Secret of Us (Deluxe),166300,2024-10-18
4,7tI8dRuH2Yc6RuoTjxo4dU,Who,Jimin,MUSE,170887,2024-07-19


In [88]:
database.to_csv('data.csv', index=False)